# Testing the models using the tools developed

In [1]:
# Load the required modules
import sys
sys.path.append("../..")
from datasetGenerator import DCASE2018
from Encoder import Encoder
from Binarizer import Binarizer

from keras.layers import GRU, Bidirectional, Layer, TimeDistributed, Dense, GRUCell
from keras.models import model_from_json, Model
from keras import backend as K
from keras import regularizers
import matplotlib.pyplot as plt
import matplotlib.patches as patch
from subprocess import call

# evaluate
from evaluation_measures import event_based_evaluation
from dcase_util.containers import MetaDataContainer

import librosa.display
import librosa
import numpy as np
import os
import copy

Using TensorFlow backend.


# Load the dataset

In [2]:
# Load the test data create the test dataset
# load the file list
featTestPath = "/homeLocal/eriador/Documents/Corpus/DCASE2018/features_2/test/mel"
featTestList = os.listdir(featTestPath)

# load the meta data ----
metaPath = "/homeLocal/eriador/Documents/Corpus/DCASE2018/meta/test.csv"
with open(metaPath, "r") as metaFile:
    metadata = metaFile.read().splitlines()[1:]
    
metadata = [i.split("\t") for i in metadata]

# load the features
featTest = []
for file in featTestList:
    path = os.path.join(featTestPath, file)
    feature = np.load(path)
    
    # preprocessing
    feature = np.expand_dims(feature, axis=-1)
    featTest.append(feature)
    
featTest = np.array(featTest)

# Load the models

In [3]:
modelJsonPath = "/homeLocal/eriador/Documents/DCASE2018/results/testing/mel_old_noBlank_timeReduction2/oldMel_noBlank_reduce2_model.json"
modelWeightPath = "/homeLocal/eriador/Documents/DCASE2018/results/testing/mel_old_noBlank_timeReduction2/oldMel_noBlank_reduce2_weight.h5py"

# Custom implementation for GRU and GRUCell

In [4]:
class CustomGRUCell(GRUCell):

    def __init__(self, units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True,
                 kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros',
                 kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, kernel_constraint=None,
                 recurrent_constraint=None, bias_constraint=None, dropout=0., recurrent_dropout=0., implementation=1,
                 reset_after=False, temporal_weight: float = 0.5, **kwargs):

        self.temporal_weight = temporal_weight

        super().__init__(units, activation, recurrent_activation, use_bias, kernel_initializer, recurrent_initializer,
                         bias_initializer, kernel_regularizer, recurrent_regularizer, bias_regularizer,
                         kernel_constraint, recurrent_constraint, bias_constraint, dropout, recurrent_dropout,
                         implementation, reset_after, **kwargs)
        
        print("Temporal weight : ", self.temporal_weight)


    def call(self, inputs, states, training=None):
        h_tm1 = states[0]  # previous memory

        # if 0 < self.dropout < 1 and self._dropout_mask is None:
        #     self._dropout_mask = _generate_dropout_mask(
        #         K.ones_like(inputs),
        #         self.dropout,
        #         training=training,
        #         count=3)
        # if (0 < self.recurrent_dropout < 1 and
        #         self._recurrent_dropout_mask is None):
        #     self._recurrent_dropout_mask = _generate_dropout_mask(
        #         K.ones_like(h_tm1),
        #         self.recurrent_dropout,
        #         training=training,
        #         count=3)

        # dropout matrices for input units
        dp_mask = self._dropout_mask
        # dropout matrices for recurrent units
        rec_dp_mask = self._recurrent_dropout_mask

        if self.implementation == 1:
            if 0. < self.dropout < 1.:
                inputs_z = inputs * dp_mask[0]
                inputs_r = inputs * dp_mask[1]
                inputs_h = inputs * dp_mask[2]
            else:
                inputs_z = inputs
                inputs_r = inputs
                inputs_h = inputs

            x_z = K.dot(inputs_z, self.kernel_z)
            x_r = K.dot(inputs_r, self.kernel_r)
            x_h = K.dot(inputs_h, self.kernel_h)
            if self.use_bias:
                x_z = K.bias_add(x_z, self.input_bias_z)
                x_r = K.bias_add(x_r, self.input_bias_r)
                x_h = K.bias_add(x_h, self.input_bias_h)

                
                
                
                
                
                
                
                
            if 0. < self.recurrent_dropout < 1.:
                h_tm1_z = h_tm1 * self.temporal_weight #rec_dp_mask[0]
                h_tm1_r = h_tm1 * self.temporal_weight #rec_dp_mask[1]
                h_tm1_h = h_tm1 * self.temporal_weight #rec_dp_mask[2]
            else:
                h_tm1_z = h_tm1 * self.temporal_weight
                h_tm1_r = h_tm1 * self.temporal_weight
                h_tm1_h = h_tm1 * self.temporal_weight

                
                
                
                
                
                
            recurrent_z = K.dot(h_tm1_z, self.recurrent_kernel_z)
            recurrent_r = K.dot(h_tm1_r, self.recurrent_kernel_r)
            if self.reset_after and self.use_bias:
                recurrent_z = K.bias_add(recurrent_z, self.recurrent_bias_z)
                recurrent_r = K.bias_add(recurrent_r, self.recurrent_bias_r)

            z = self.recurrent_activation(x_z + recurrent_z)
            r = self.recurrent_activation(x_r + recurrent_r)

            # reset gate applied after/before matrix multiplication
            if self.reset_after:
                recurrent_h = K.dot(h_tm1_h, self.recurrent_kernel_h)
                if self.use_bias:
                    recurrent_h = K.bias_add(recurrent_h, self.recurrent_bias_h)
                recurrent_h = r * recurrent_h
            else:
                recurrent_h = K.dot(r * h_tm1_h, self.recurrent_kernel_h)

            hh = self.activation(x_h + recurrent_h)
        else:
            if 0. < self.dropout < 1.:
                inputs *= dp_mask[0]

            # inputs projected by all gate matrices at once
            matrix_x = K.dot(inputs, self.kernel)
            if self.use_bias:
                # biases: bias_z_i, bias_r_i, bias_h_i
                matrix_x = K.bias_add(matrix_x, self.input_bias)
            x_z = matrix_x[:, :self.units]
            x_r = matrix_x[:, self.units: 2 * self.units]
            x_h = matrix_x[:, 2 * self.units:]

            if 0. < self.recurrent_dropout < 1.:
                h_tm1 *= rec_dp_mask[0]

            if self.reset_after:
                # hidden state projected by all gate matrices at once
                matrix_inner = K.dot(h_tm1, self.recurrent_kernel)
                if self.use_bias:
                    matrix_inner = K.bias_add(matrix_inner, self.recurrent_bias)
            else:
                # hidden state projected separately for update/reset and new
                matrix_inner = K.dot(h_tm1,
                                     self.recurrent_kernel[:, :2 * self.units])

            recurrent_z = matrix_inner[:, :self.units]
            recurrent_r = matrix_inner[:, self.units: 2 * self.units]

            z = self.recurrent_activation(x_z + recurrent_z)
            r = self.recurrent_activation(x_r + recurrent_r)

            if self.reset_after:
                recurrent_h = r * matrix_inner[:, 2 * self.units:]
            else:
                recurrent_h = K.dot(r * h_tm1,
                                    self.recurrent_kernel[:, 2 * self.units:])

            hh = self.activation(x_h + recurrent_h)

        # previous and candidate state mixed by update gate
        h = z * h_tm1 + (1 - z) * hh

        if 0 < self.dropout + self.recurrent_dropout:
            if training is None:
                h._uses_learning_phase = True

        return h, [h]


class CustomGRU(GRU):

    def __init__(self, units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True,
                 kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros',
                 kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None,
                 kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.,
                 recurrent_dropout=0., implementation=1, return_sequences=False, return_state=False, go_backwards=False,
                 stateful=False, unroll=False, reset_after=False, temporal_weight: float = 0.5, **kwargs):

        """
        super().__init__(units, activation=activation, recurrent_activation=recurrent_activation,
                         use_bias=use_bias, kernel_initializer=kernel_initializer, recurrent_initializer=recurrent_initializer,
                         bias_initializer=bias_initializer, kernel_regularizer=kernel_regularizer,
                         recurrent_regularizer=recurrent_regularizer, bias_regularizer=bias_regularizer,
                         activity_regularizer=activity_regularizer, kernel_constraint=kernel_constraint,
                         recurrent_constraint=recurrent_constraint, bias_constraint=bias_constraint,
                         dropout=dropout, recurrent_dropout=recurrent_dropout, implementation=implementation,
                         return_sequences=return_sequences, return_state=return_state, go_backwards=go_backwards,
                         stateful=stateful, unroll=unroll, reset_after=reset_after, **kwargs)
        """
        
        self.temporal_weight = temporal_weight

        cell = CustomGRUCell(units,
                       activation=activation,
                       recurrent_activation=recurrent_activation,
                       use_bias=use_bias,
                       kernel_initializer=kernel_initializer,
                       recurrent_initializer=recurrent_initializer,
                       bias_initializer=bias_initializer,
                       kernel_regularizer=kernel_regularizer,
                       recurrent_regularizer=recurrent_regularizer,
                       bias_regularizer=bias_regularizer,
                       kernel_constraint=kernel_constraint,
                       recurrent_constraint=recurrent_constraint,
                       bias_constraint=bias_constraint,
                       dropout=dropout,
                       recurrent_dropout=recurrent_dropout,
                       implementation=implementation,
                       reset_after=reset_after,
                       temporal_weight=temporal_weight)

        super(GRU, self).__init__(cell,
                                  return_sequences=return_sequences,
                                  return_state=return_state,
                                  go_backwards=go_backwards,
                                  stateful=stateful,
                                  unroll=unroll,
                                  **kwargs)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        
        
        
    def get_config(self):
        config = super().get_config()
        config["temporal_weight"] = self.temporal_weight
        return config
        
    def call(self, inputs, mask=None, training=None, initial_state=None):
        return super().call(inputs, mask, True, initial_state)
        
    

# With GRU and TimeDistributed
## Remove the GlobalAveragePooling1D to trace score per frames

In [5]:
##### load the model and perform prediction
def useClassic():
    K.clear_session()

    with open(modelJsonPath, "r") as modelJsonFile:
        model = model_from_json(modelJsonFile.read())
    model.load_weights(modelWeightPath)

    #model.summary()

    intermediate_model = Model(input=model.input, output=model.get_layer("time_distributed_1").output)
    #intermediate_model.summary()
    
    return intermediate_model

# retreive information about the custom
#model = loadClassic()
#prediction = model.predict(featTest)

# With ***Custom*** GRU and TimeDistributed
## Remove the GlobalAveragePooling1D to trace score per frames

In [6]:
##### load the model and perform prediction
def useCustomGRU(temporalWeight: float) -> Model:
    K.clear_session()
    
    with open(modelJsonPath, "r") as modelJsonFile:
        model = model_from_json(modelJsonFile.read())
    model.load_weights(modelWeightPath)

    #disasemble layers
    layers = [l for l in model.layers]

    # Get the trained forward layer from the bidirectional and change it's property
    b1 = model.get_layer("bidirectional_1")

    x = layers[0].output
    for i in range(1, len(layers)):
        if layers[i].name == "bidirectional_1":
            x = Bidirectional(
                CustomGRU(units=64, kernel_initializer='glorot_uniform', recurrent_dropout=0.8, dropout=0.0, return_sequences=True, temporal_weight=temporalWeight), name="custom_bi")(x)
        elif layers[i].name == "time_distributed_1":
            x = TimeDistributed( Dense(10, activation="sigmoid"), )(x)
        else:
            x = layers[i](x)

    newModel = Model(input=layers[0].input, output=x)
    newModel.load_weights(modelWeightPath)
    #model.summary()

    intermediate_model = Model(input=model.input, output=newModel.get_layer("time_distributed_1").output)
    #intermediate_model.summary()
    
    return intermediate_model

# retreive inform,ation about the custom
#model = useCustomGRU(0.2)
#prediction = model.predict(featTest)

# Calculate the score using the **baseline tool**

In [7]:
def basic(method: str = "threshold", **kwargs):
    e = Encoder()
    segments = e.encode(prediction, method=method, **kwargs)
    evaluation = e.parse(segments, featTestList)
    
    # write the evaluation on the disk
    with open("perso_eval.csv", "w") as f:
        f.write("filename\tonset\toffset\tevent_label\n")
        f.write(evaluation)

    perso_event_list = MetaDataContainer()
    perso_event_list.load(filename="perso_eval.csv")

    ref_event_list = MetaDataContainer()
    ref_event_list.load(filename="../../meta/test.csv")

    event_based_metric = event_based_evaluation(ref_event_list, perso_event_list)

    print(event_based_metric)
    
def printTable(results: list):
    
    print("%-*s | %-*s | %-*s | %-*s | %-*s | %-*s | %-*s | %-*s | %-*s |" % (
        22, "model", 22, "Global F-measure", 22, "Global Precision", 22, "Global Recall", 22, "ER",
        22, "Class-wise F-measure", 22, "Class-wise Precision", 22, "Class-wise Recall", 22, "ER"))
    print(("-" * 22 + " | ") * 9)
    
    for weight, info in results:
        print("%-*s | " % (22, "CGRU %.2f" % (weight)), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["overall"]["f_measure"]["f_measure"] * 100)), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["overall"]["f_measure"]["precision"] * 100)), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["overall"]["f_measure"]["recall"] * 100)), end="")
        print("%-*s | " % (22, "%.2f " % (info["overall"]["error_rate"]["error_rate"])), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["class-wise"]["f_measure"]["f_measure"] * 100)), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["class-wise"]["f_measure"]["precision"] * 100)), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["class-wise"]["f_measure"]["recall"] * 100)), end="")
        print("%-*s | " % (22, "%.2f " % (info["class-wise"]["error_rate"]["error_rate"])), end="")
        print("")
        
def evaluate(evaluation):
    # write the evaluation on the disk
    print("perform evaluation ...")
    with open("perso_eval.csv", "w") as f:
        f.write("filename\tonset\toffset\tevent_label\n")
        f.write(evaluation)

    perso_event_list = MetaDataContainer()
    perso_event_list.load(filename="perso_eval.csv")

    ref_event_list = MetaDataContainer()
    ref_event_list.load(filename="../../meta/test.csv")

    event_based_metric = event_based_evaluation(ref_event_list, perso_event_list)
    #print(event_based_metric)

    result = {
        "overall": event_based_metric.results_overall_metrics(),
        "class-wise": event_based_metric.results_class_wise_average_metrics(),
    }

    return result
    

def advanced(temporal_weights: list, method="threshold", **kwargs) -> list:
    results = []
    
    # classic ----------------------
    print("loading model ...")
    model = useClassic()
    print("predicting ...")
    prediction = model.predict(featTest)
    
    e = Encoder()
    print("encoding results ...")
    segments = e.encode(prediction, method=method, **kwargs)
    evaluation = e.parse(segments, featTestList)
    
    results.append((1.0, evaluate(evaluation)))
    
    # custom GRU prediction - ---------------------
    for weight in temporal_weights:
        print("loading model ...")
        model = useCustomGRU(weight)
        print("predicting ...")
        prediction = model.predict(featTest)
        
        e = Encoder()
        print("encoding results ...")
        segments = e.encode(prediction, method=method, **kwargs)
        evaluation = e.parse(segments, featTestList)
        
        results.append((weight, evaluate(evaluation)))
        
    return results
        
#print("HYSTERESIS")
#printTable( advanced([0.1, 0.2, 0.23, 0.25, 0.3, 0.5], method="hysteresis") )

#print("PRIMITIVE")
#printTable( advanced([0.1, 0.2, 0.23, 0.25, 0.3, 0.5], method="primitive", window_size=10, stride=1, thresshold=3) )

print("THRESHOLD")
printTable( advanced([0.1, 0.2, 0.23, 0.25, 0.3, 0.5]) )

#basic(method="hysteresis")
#basic(method="hysteresis", high=0.7, low=0.3)
#basic(method="hysteresis", high=0.8, low=0.2)
#basic(method="derivative")
#basic(method="primitive", window_size=10, stride=1, threshold=3)
#basic()

THRESHOLD
loading model ...


/homeLocal/eriador/.miniconda3/envs/dcase/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ti...)`
  # This is added back by InteractiveShellApp.init_path()


predicting ...
encoding results ...
perform evaluation ...
loading model ...
Temporal weight :  0.1
Temporal weight :  0.1


/homeLocal/eriador/.miniconda3/envs/dcase/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`
/homeLocal/eriador/.miniconda3/envs/dcase/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ti...)`


predicting ...
encoding results ...
perform evaluation ...
loading model ...
Temporal weight :  0.2
Temporal weight :  0.2
predicting ...
encoding results ...
perform evaluation ...
loading model ...
Temporal weight :  0.23
Temporal weight :  0.23
predicting ...
encoding results ...
perform evaluation ...
loading model ...
Temporal weight :  0.25
Temporal weight :  0.25
predicting ...
encoding results ...
perform evaluation ...
loading model ...
Temporal weight :  0.3
Temporal weight :  0.3
predicting ...
encoding results ...
perform evaluation ...
loading model ...
Temporal weight :  0.5
Temporal weight :  0.5
predicting ...
encoding results ...
perform evaluation ...
model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ----------------------

In [ ]:
# evaluate
from evaluation_measures import event_based_evaluation
from dcase_util.containers import MetaDataContainer

# write the evaluation on the disk
with open("perso_eval.csv", "w") as f:
    f.write("filename\tonset\toffset\tevent_label\n")
    f.write(evaluation)
    
perso_event_list = MetaDataContainer()
perso_event_list.load(filename="perso_eval.csv")
GRU | 3.79 % | 3.15 % | 4.75 % | - | 4.96 % | 3.56 % | 11.99 % | - |
ref_event_list = MetaDataContainer()
ref_event_list.load(filename="../../meta/test.csv")

event_based_metric = event_based_evaluation(ref_event_list, perso_event_list)

print(event_based_metric)

# Different segmentation methods results for "old mel without blank" and 20 ms frame length

## *Threshold based segmentation with "hole filling"*

 - **CGRU x.xx** : Custom Gate Recurent Unit usage with temporal weight of x.xx
 - **GRU** : The classic GRU implementation
 
model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- |
GRU                    | 3.79 %                 | 3.15 %                 | 4.75 %                 | 2.38                   | 4.96 %                 | 3.56 %                 | 11.99 %                | 3.25                   | 
CGRU 0.10              | 6.61 %                 | 4.88 %                 | 10.26 %                | 2.88                   | 5.55 %                 | 6.68 %                 | 7.25 %                 | 5.18                   | 
CGRU 0.20              | 9.60 %                 | 7.19 %                 | 14.46 %                | 2.70                   | 8.25 %                 | 9.82 %                 | 10.43 %                | 4.78                   | 
CGRU 0.23              | 10.41 %                | 7.88 %                 | 15.34 %                | 2.62                   | 9.40 %                 | 10.85 %                | 11.86 %                | 4.61                   | 
CGRU 0.25              | 10.59 %                | 8.06 %                 | 15.45 %                | 2.59                   | 9.88 %                 | 16.22 %                | 12.17 %                | 4.50                   | 
CGRU 0.30              | 9.45 %                 | 7.32 %                 | 13.36 %                | 2.54                   | 9.23 %                 | 10.04 %                | 12.03 %                | 4.30                   | 
CGRU 0.50              | 4.23 %                 | 3.39 %                 | 5.63 %                 | 2.52                   | 4.47 %                 | 4.07 %                 | 9.26 %                 | 3.68                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Hysteresis based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 4.52 %                 | 4.40 %                 | 4.64 %                 | 1.93                   | 6.40 %                 | 5.05 %                 | 11.75 %                | 2.48                   | 
CGRU 0.10              | 2.94 %                 | 2.26 %                 | 4.19 %                 | 2.76                   | 2.27 %                 | 2.02 %                 | 3.60 %                 | 4.55                   | 
CGRU 0.20              | 5.05 %                 | 4.04 %                 | 6.73 %                 | 2.52                   | 4.61 %                 | 4.98 %                 | 5.72 %                 | 4.17                   | 
CGRU 0.23              | 5.40 %                 | 4.37 %                 | 7.06 %                 | 2.46                   | 4.44 %                 | 5.00 %                 | 5.82 %                 | 4.04                   | 
CGRU 0.25              | 5.41 %                 | 4.42 %                 | 6.95 %                 | 2.43                   | 4.37 %                 | 5.13 %                 | 5.83 %                 | 3.93                   | 
CGRU 0.30              | 5.93 %                 | 5.00 %                 | 7.28 %                 | 2.30                   | 5.52 %                 | 7.95 %                 | 6.75 %                 | 3.67                   | 
CGRU 0.50              | 3.60 %                 | 3.23 %                 | 4.08 %                 | 2.15                   | 3.83 %                 | 3.51 %                 | 7.90 %                 | 3.06                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Primitive based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 5.11 %                 | 4.25 %                 | 6.40 %                 | 2.33                   | 6.29 %                 | 4.60 %                 | 15.28 %                | 3.22                   | 
CGRU 0.10              | 7.46 %                 | 5.48 %                 | 11.70 %                | 2.88                   | 7.13 %                 | 8.26 %                 | 9.00 %                 | 5.21                   | 
CGRU 0.20              | 9.29 %                 | 6.95 %                 | 14.02 %                | 2.72                   | 9.14 %                 | 13.17 %                | 12.08 %                | 4.70                   | 
CGRU 0.23              | 9.19 %                 | 6.91 %                 | 13.69 %                | 2.69                   | 9.12 %                 | 10.09 %                | 12.34 %                | 4.59                   | 
CGRU 0.25              | 8.59 %                 | 6.53 %                 | 12.58 %                | 2.65                   | 8.79 %                 | 9.25 %                 | 12.07 %                | 4.45                   | 
CGRU 0.30              | 6.25 %                 | 4.77 %                 | 9.05 %                 | 2.69                   | 6.76 %                 | 6.77 %                 | 9.82 %                 | 4.31                   | 
CGRU 0.50              | 3.63 %                 | 2.89 %                 | 4.86 %                 | 2.55                   | 4.18 %                 | 3.67 %                 | 9.91 %                 | 3.72                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |


# Different segmentation methods results for "old mel without blank" and **40** ms frame length

## *Threshold based segmentation with "hole filling"*

 - **CGRU x.xx** : Custom Gate Recurent Unit usage with temporal weight of x.xx
 - **GRU** : The classic GRU implementation
 
model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- |
GRU                    | 3.79 %                 | 3.15 %                 | 4.75 %                 | 2.38                   | 4.96 %                 | 3.56 %                 | 11.99 %                | 3.25                   | 
CGRU 0.10              | 6.61 %                 | 4.88 %                 | 10.26 %                | 2.88                   | 5.55 %                 | 6.68 %                 | 7.25 %                 | 5.18                   | 
CGRU 0.20              | 9.60 %                 | 7.19 %                 | 14.46 %                | 2.70                   | 8.25 %                 | 9.82 %                 | 10.43 %                | 4.78                   | 
CGRU 0.23              | 10.41 %                | 7.88 %                 | 15.34 %                | 2.62                   | 9.40 %                 | 10.85 %                | 11.86 %                | 4.61                   | 
CGRU 0.25              | 10.59 %                | 8.06 %                 | 15.45 %                | 2.59                   | 9.88 %                 | 16.22 %                | 12.17 %                | 4.50                   | 
CGRU 0.30              | 9.45 %                 | 7.32 %                 | 13.36 %                | 2.54                   | 9.23 %                 | 10.04 %                | 12.03 %                | 4.30                   | 
CGRU 0.50              | 4.23 %                 | 3.39 %                 | 5.63 %                 | 2.52                   | 4.47 %                 | 4.07 %                 | 9.26 %                 | 3.68                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Hysteresis based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 4.52 %                 | 4.40 %                 | 4.64 %                 | 1.93                   | 6.40 %                 | 5.05 %                 | 11.75 %                | 2.48                   | 
CGRU 0.10              | 2.94 %                 | 2.26 %                 | 4.19 %                 | 2.76                   | 2.27 %                 | 2.02 %                 | 3.60 %                 | 4.55                   | 
CGRU 0.20              | 5.05 %                 | 4.04 %                 | 6.73 %                 | 2.52                   | 4.61 %                 | 4.98 %                 | 5.72 %                 | 4.17                   | 
CGRU 0.23              | 5.40 %                 | 4.37 %                 | 7.06 %                 | 2.46                   | 4.44 %                 | 5.00 %                 | 5.82 %                 | 4.04                   | 
CGRU 0.25              | 5.41 %                 | 4.42 %                 | 6.95 %                 | 2.43                   | 4.37 %                 | 5.13 %                 | 5.83 %                 | 3.93                   | 
CGRU 0.30              | 5.93 %                 | 5.00 %                 | 7.28 %                 | 2.30                   | 5.52 %                 | 7.95 %                 | 6.75 %                 | 3.67                   | 
CGRU 0.50              | 3.60 %                 | 3.23 %                 | 4.08 %                 | 2.15                   | 3.83 %                 | 3.51 %                 | 7.90 %                 | 3.06                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Primitive based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 5.11 %                 | 4.25 %                 | 6.40 %                 | 2.33                   | 6.29 %                 | 4.60 %                 | 15.28 %                | 3.22                   | 
CGRU 0.10              | 7.46 %                 | 5.48 %                 | 11.70 %                | 2.88                   | 7.13 %                 | 8.26 %                 | 9.00 %                 | 5.21                   | 
CGRU 0.20              | 9.29 %                 | 6.95 %                 | 14.02 %                | 2.72                   | 9.14 %                 | 13.17 %                | 12.08 %                | 4.70                   | 
CGRU 0.23              | 9.19 %                 | 6.91 %                 | 13.69 %                | 2.69                   | 9.12 %                 | 10.09 %                | 12.34 %                | 4.59                   | 
CGRU 0.25              | 8.59 %                 | 6.53 %                 | 12.58 %                | 2.65                   | 8.79 %                 | 9.25 %                 | 12.07 %                | 4.45                   | 
CGRU 0.30              | 6.25 %                 | 4.77 %                 | 9.05 %                 | 2.69                   | 6.76 %                 | 6.77 %                 | 9.82 %                 | 4.31                   | 
CGRU 0.50              | 3.63 %                 | 2.89 %                 | 4.86 %                 | 2.55                   | 4.18 %                 | 3.67 %                 | 9.91 %                 | 3.72                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |


## **results - old Mel without blank class, 80 ms frame**
##### *Threshold based segmentation with "hole filling"*

 - **CGRU x.xx** : Custom Gate Recurent Unit usage with temporal weight of x.xx
 - **GRU** : The classic GRU implementation
 
model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
CGRU 0.1000            | 0.1170                 | 0.1306                 | 0.1060                 | 1.5872                 | 0.0887                 | 0.1075                 | 0.0677                 | 1.9875                 | 
CGRU 0.2000            | 0.0980                 | 0.1069                 | 0.0905                 | 1.6468                 | 0.0785                 | 0.0915                 | 0.0708                 | 2.0738                 | 
CGRU 0.2300            | 0.0785                 | 0.0867                 | 0.0717                 | 1.6678                 | 0.0684                 | 0.0849                 | 0.0606                 | 2.0939                 | 
CGRU 0.2500            | 0.0736                 | 0.0812                 | 0.0673                 | 1.6766                 | 0.0697                 | 0.0834                 | 0.0690                 | 2.0877                 | 
CGRU 0.3000            | 0.0647                 | 0.0678                 | 0.0618                 | 1.7627                 | 0.0584                 | 0.0697                 | 0.0643                 | 2.2389                 | 
CGRU 0.5000            | 0.0536                 | 0.0532                 | 0.0541                 | 1.8698                 | 0.0635                 | 0.0611                 | 0.0910                 | 2.3879                 | 
GRU | 3.79 % | 3.15 % | 4.75 % | 4.96 % | 3.56 % | 11.99 % |
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |


## **results - old Mel without blank class, 160 ms frame**
##### *Threshold based segmentation with "hole filling"*

 - **CGRU x.xx** : Custom Gate Recurent Unit usage with temporal weight of x.xx
 - **GRU** : The classic GRU implementation
 
model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
CGRU 0.1000            | 0.0794                 | 0.0945                 | 0.0684                 | 1.5662                 | 0.0793                 | 0.1106                 | 0.0629                 | 2.0804                 | 
CGRU 0.2000            | 0.0691                 | 0.0801                 | 0.0607                 | 1.6148                 | 0.0622                 | 0.0859                 | 0.0618                 | 2.1476                 | 
CGRU 0.2300            | 0.0667                 | 0.0758                 | 0.0596                 | 1.6391                 | 0.0619                 | 0.0822                 | 0.0656                 | 2.1699                 | 
CGRU 0.2500            | 0.0733                 | 0.0820                 | 0.0662                 | 1.6490                 | 0.0692                 | 0.0920                 | 0.0716                 | 2.1479                 | 
CGRU 0.3000            | 0.0580                 | 0.0640                 | 0.0530                 | 1.6932                 | 0.0565                 | 0.0710                 | 0.0662                 | 2.1601                 | 
CGRU 0.5000            | 0.0505                 | 0.0571                 | 0.0453                 | 1.6744                 | 0.0647                 | 0.0682                 | 0.0915                 | 1.9955                 | 
GRU | 3.79 % | 3.15 % | 4.75 % | 4.96 % | 3.56 % | 11.99 % |
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |
